# The JOIN operation

![rel](https://sqlzoo.net/w/images/c/ce/FootballERD.png)

## JOIN and UEFA EURO 2012

This tutorial introduces `JOIN` which allows you to use data from two or more tables. The tables contain all matches and goals from UEFA EURO 2012 Football Championship in Poland and Ukraine.

The data is available (mysql format) at <http://sqlzoo.net/euro2012.sql>

In [1]:
import os
import pandas as pd
import findspark
os.environ['SPARK_HOME'] =  '/opt/spark'
findspark.init()

from pyspark.sql import SparkSession
sc = (SparkSession.builder.appName('app06')
      .config('spark.sql.warehouse.dir', 'hdfs://quickstart.cloudera:8020/user/hive/warehouse')
      .config('hive.metastore.uris', 'thrift://quickstart.cloudera:9083')
      .enableHiveSupport().getOrCreate())

## 1.
The first example shows the goal scored by a player with the last name 'Bender'. The `*` says to list all the columns in the table - a shorter way of saying `matchid, teamid, player, gtime`

**Modify it to show the matchid and player name for all goals scored by Germany. To identify German players, check for: `teamid = 'GER'`**

In [2]:
game = sc.read.table('sqlzoo.game')
goal = sc.read.table('sqlzoo.goal')
eteam = sc.read.table('sqlzoo.eteam')

In [3]:
goal.filter(goal['teamid']=='GER').select('matchid', 'player').toPandas()

,matchid,player
0,1008,Mario Gómez
1,1010,Mario Gómez
2,1010,Mario Gómez
3,1012,Lukas Podolski
4,1012,Lars Bender
5,1026,Philipp Lahm
6,1026,Sami Khedira
7,1026,Miroslav Klose
8,1026,Marco Reus
9,1030,Mesut Özil


## 2.

From the previous query you can see that Lars Bender's scored a goal in game 1012. Now we want to know what teams were playing in that match.

Notice in the that the column `matchid `in the `goal` table corresponds to the `id` column in the `game` table. We can look up information about game 1012 by finding that row in the `game` table.

**Show id, stadium, team1, team2 for just game 1012**

In [4]:
game.filter(game['id']==1012).select('id', 'stadium', 'team1', 'team2').toPandas()

,id,stadium,team1,team2
0,1012,Arena Lviv,DEN,GER


## 3.
You can combine the two steps into a single query with a JOIN.

```sql
SELECT *
  FROM game JOIN goal ON (id=matchid)
```

The **FROM** clause says to merge data from the goal table with that from the game table. The **ON** says how to figure out which rows in **game** go with which rows in **goal** - the **matchid** from **goal** must match **id** from **game**. (If we wanted to be more clear/specific we could say
`ON (game.id=goal.matchid)`

The code below shows the player (from the goal) and stadium name (from the game table) for every goal scored.

**Modify it to show the player, teamid, stadium and mdate for every German goal.**

In [5]:
(game.join(goal, game['id']==goal['matchid'])
    .filter(goal['teamid']=='GER')
    .select('player', 'teamid', 'stadium', 'mdate')
    .toPandas())

,player,teamid,stadium,mdate
0,Mario Gómez,GER,Arena Lviv,9 June 2012
1,Mario Gómez,GER,Metalist Stadium,13 June 2012
2,Mario Gómez,GER,Metalist Stadium,13 June 2012
3,Lukas Podolski,GER,Arena Lviv,17 June 2012
4,Lars Bender,GER,Arena Lviv,17 June 2012
5,Philipp Lahm,GER,PGE Arena Gdansk,22 June 2012
6,Sami Khedira,GER,PGE Arena Gdansk,22 June 2012
7,Miroslav Klose,GER,PGE Arena Gdansk,22 June 2012
8,Marco Reus,GER,PGE Arena Gdansk,22 June 2012
9,Mesut Özil,GER,"National Stadium, Warsaw",28 June 2012


## 4.
Use the same `JOIN` as in the previous question.

**Show the team1, team2 and player for every goal scored by a player called Mario `player LIKE 'Mario%'`**

In [6]:
(game.join(goal, game['id']==goal['matchid'])
    .filter(goal['player'].startswith('Mario'))
    .select('team1', 'team2', 'player').toPandas())

,team1,team2,player
0,GER,POR,Mario Gómez
1,NED,GER,Mario Gómez
2,NED,GER,Mario Gómez
3,IRL,CRO,Mario Mandžukic
4,IRL,CRO,Mario Mandžukic
5,ITA,CRO,Mario Mandžukic
6,ITA,IRL,Mario Balotelli
7,GER,ITA,Mario Balotelli
8,GER,ITA,Mario Balotelli


## 5.

The table `eteam` gives details of every national team including the coach. You can `JOIN` `goal` to `eteam` using the phrase goal `JOIN eteam on teamid=id`

**Show `player, teamid, coach, gtime` for all goals scored in the first 10 minutes `gtime<=10`**

In [7]:
(goal.join(eteam, goal['teamid']==eteam['id'])
    .filter(goal.gtime<=10)
    .select('player', 'teamid', 'coach', 'gtime')
    .toPandas())

,player,teamid,coach,gtime
0,Petr Jirácek,CZE,Michal Bílek,3
1,Václav Pilar,CZE,Michal Bílek,6
2,Mario Mandžukic,CRO,Slaven Bilic,3
3,Fernando Torres,ESP,Vicente del Bosque,4


## 6.

To `JOIN` `game` with `eteam` you could use either
`game JOIN eteam ON (team1=eteam.id)` or `game JOIN eteam ON (team2=eteam.id)`

Notice that because `id` is a column name in both `game` and `eteam` you must specify `eteam.id` instead of just `id`

**List the the dates of the matches and the name of the team in which 'Fernando Santos' was the team1 coach.**

In [8]:
(game.join(eteam, game['team1']==eteam['id'])
 .filter("coach=='Fernando Santos'")
 .select('mdate', 'teamname')
 .toPandas())

,mdate,teamname
0,12 June 2012,Greece
1,16 June 2012,Greece


## 7.

**List the player for every goal scored in a game where the stadium was 'National Stadium, Warsaw'**

In [9]:
(goal.join(game, goal['matchid']==game['id'])
 .filter("stadium=='National Stadium, Warsaw'")
 .select('player').toPandas())

,player
0,Robert Lewandowski
1,Dimitris Salpingidis
2,Alan Dzagoev
3,Jakub Blaszczykowski
4,Giorgos Karagounis
5,Cristiano Ronaldo
6,Mario Balotelli
7,Mario Balotelli
8,Mesut Özil


## 8. More difficult questions

The example query shows all goals scored in the Germany-Greece quarterfinal.
**Instead show the name of all players who scored a goal against Germany.**

> __HINT__   
> Select goals scored only by non-German players in matches where GER was the id of either **team1** or **team2**.
> You can use `teamid!='GER'` to prevent listing German players.
> You can use `DISTINCT` to stop players being listed twice.

In [10]:
(game.join(goal, game['id']==goal['matchid'])
 .filter("(team1 == 'GER' or team2 == 'GER') and (teamid != 'GER')")
 .select('player').dropDuplicates()
 .toPandas())

,player
0,Michael Krohn-Dehli
1,Robin van Persie
2,Mario Balotelli
3,Dimitris Salpingidis
4,Georgios Samaras


## 9.
Show teamname and the total number of goals scored.

> __COUNT and GROUP BY__  
> You should COUNT(*) in the SELECT line and GROUP BY teamname

In [11]:
(eteam.join(goal, eteam['id']==goal['teamid'])
 .select('teamname', 'player')
 .groupBy('teamname')
 .count().toPandas())

,teamname,count
0,Russia,5
1,Sweden,5
2,Germany,10
3,France,3
4,Greece,5
5,Croatia,4
6,Italy,6
7,Spain,12
8,Denmark,4
9,Ukraine,2


## 10.

**Show the stadium and the number of goals scored in each stadium.**

In [12]:
(game.join(goal, game['id']==goal['matchid'])
 .select('stadium', 'player')
 .groupBy('stadium')
 .count().toPandas())

,stadium,count
0,Metalist Stadium,7
1,Arena Lviv,9
2,Stadion Miejski (Poznan),8
3,Donbass Arena,7
4,Stadion Miejski (Wroclaw),9
5,Olimpiyskiy National Sports Complex,14
6,"National Stadium, Warsaw",9
7,PGE Arena Gdansk,13


## 11.
**For every match involving 'POL', show the matchid, date and the number of goals scored.**

In [13]:
(game.join(goal, game['id']==goal['matchid'])
 .filter("team1=='POL' or team2=='POL'")
 .select('matchid', 'mdate', 'player')
 .groupBy('matchid', 'mdate')
 .count().toPandas())

,matchid,mdate,count
0,1004,12 June 2012,2
1,1001,8 June 2012,2
2,1005,16 June 2012,1


## 12.
**For every match where 'GER' scored, show matchid, match date and the number of goals scored by 'GER'**

In [14]:
(game.join(goal, game['id']==goal['matchid'])
 .filter("teamid=='GER'")
 .select('matchid', 'mdate', 'player')
 .groupBy('matchid', 'mdate')
 .count().toPandas())

,matchid,mdate,count
0,1012,17 June 2012,2
1,1010,13 June 2012,2
2,1030,28 June 2012,1
3,1026,22 June 2012,4
4,1008,9 June 2012,1


## 13.
List every match with the goals scored by each team as shown. This will use "CASE WHEN" which has not been explained in any previous exercises.

no  | mdate      | team1  | score1 | team2 | score2
----|------------|--------|--------|-------|--------
1   | July 2012  | ESP    | 4      | ITA   | 0
10  | June 2012  | ESP    | 1      | ITA   | 1
10  | June 2012  | IRL    | 1      | CRO   | 3
... | ...        | ...    | ...    | ...   | ...

Notice in the query given every goal is listed. If it was a team1 goal then a 1 appears in score1, otherwise there is a 0. You could SUM this column to get a count of the goals scored by team1. **Sort your result by mdate, matchid, team1 and team2.**

In [16]:
# sc.conf.set('spark.sql.crossJoin.enabled', 'true')
s = (goal.select('matchid', 'teamid', 'player')
     .groupBy('matchid', 'teamid')
     .count())

a = (game.join(s, [game['id']==s['matchid'], game['team1']==s['teamid']], how='left')
 .drop('teamid', 'matchid').withColumnRenamed('count', 'score1').alias('a'))
(a.join(s, [a['id']==s['matchid'], a['team2']==s['teamid']], how='left')
 .drop('matchid', 'teamid').withColumnRenamed('count', 'score2')
 .select('mdate', 'team1', 'score1', 'team2', 'score2')
 .fillna(0)
 .orderBy('mdate')
 .toPandas())

,mdate,team1,score1,team2,score2
0,1 July 2012,ESP,4,ITA,0
1,10 June 2012,ESP,1,ITA,1
2,10 June 2012,IRL,1,CRO,3
3,11 June 2012,FRA,1,ENG,1
4,11 June 2012,UKR,2,SWE,1
5,12 June 2012,GRE,1,CZE,2
6,12 June 2012,POL,1,RUS,1
7,13 June 2012,DEN,2,POR,3
8,13 June 2012,NED,1,GER,2
9,14 June 2012,ITA,1,CRO,1


In [17]:
sc.stop()